# Structuring End-to-End Systems

In [38]:
from openai import OpenAI

In [2]:
from openai import OpenAI

# Path to your API key file
api_key_file_path = '/Users/dylan/Documents/AIEng1/newAPI.txt'

# Read the API key from the file
with open(api_key_file_path, 'r') as file:
    de_api_key = file.read().strip()

In [3]:
client = OpenAI(api_key=de_api_key)

In [4]:
#Important to specify in the messages the response format

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", 
         "content": "How many hours to learn Spanish? and how many weeks does that take if I study 10 h/w? What level on the EU framework will I be? Please respons in json format."}
    ],
    response_format={"type": "json_object"}
)

In [5]:
print(response.choices[0].message.content)

{
  "hours_to_learn": "600 hours",
  "weeks_to_learn": "60 weeks",
  "study_hours_per_week": 10,
  "eu_framework_level": "B1"
}


## Error Handling

- Connection issues, check connection, reach out
- Resource limits errors, conflict, ratelimit -- check amount being pushed
- Authentications - 
- Bad Request - missing or invalid parameteres 


Try and except blocks

In [6]:
try:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": "List five data science professions."}
        ]
    )

except openai.AuthenticationError as e:
    print(f"OpenAI API failed to authenticate: {e}")
    pass
except openai.RateLimitError as e:
    print(f"OpenAI API request exceeded rate limit: {e}")
    pass
except Exception as e:
    print(f"Unable to generate a response. Exception: {e}")
    pass


# Batching

- Solving rate limit error 
- Setting a short pause
- Processing multiple messages in one request (batching)




In [12]:
from tenacity import(
    retry,
    stop_after_attempt,
    wait_random_exponential
)

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6)),

SyntaxError: invalid syntax (22319831.py, line 7)

### counting tokens

In [17]:
import tiktoken

client = OpenAI(api_key=de_api_key)
input_message = {"role": "user", "content": "I'd like to buy a shirt and a jacket. Can you suggest two color pairings for these items?"}

# Use tiktoken to create the encoding for your model
encoding = tiktoken.encoding_for_model("gpt-4o-mini")
# Check for the number of tokens
num_tokens = len(encoding.encode(input_message['content']))

# Run the chat completions function and print the response
if num_tokens <= 100:
    response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[input_message])
    print(response.choices[0].message.content)
else:
    print("Message exceeds token limit")

Sure! Here are two color pairings for a shirt and jacket:

1. Shirt: Light blue Jacket: Navy blue
2. Shirt: White Jacket: Olive green


# Function Calling

- We can get rid of inconsistent formats
- Calling multiple functions to provide complext systems
- Calling external APIs (weather etc.)

In [19]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "user",
        "content": "Quickest way to learn spanish? repsond in json format"}
        
    ],
    response_format={"type": "json_object"}
)
print(response.choices[0].message.content)

{
  "quickest_way_to_learn_spanish": {
    "methods": [
      {
        "method": "Immersion",
        "description": "Surround yourself with Spanish speakers and consume Spanish media (films, music, books) to enhance listening and speaking skills."
      },
      {
        "method": "Language Apps",
        "description": "Use apps like Duolingo, Babbel, or Rosetta Stone for structured learning and daily practice."
      },
      {
        "method": "Online Courses",
        "description": "Enroll in online courses or tutorials that focus on conversational Spanish and grammar."
      },
      {
        "method": "Language Exchange",
        "description": "Partner with a native Spanish speaker who wants to learn your language for mutual practice."
      },
      {
        "method": "Flashcards",
        "description": "Utilize flashcards to build vocabulary and reinforce memory through repetition."
      },
      {
        "method": "Consistent Practice",
        "description": "Set a

In [27]:
client = OpenAI(api_key=de_api_key)

message_listings = [{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}, {'role': 'user', 'content': '\nA. M. Turing (1950) Computing Machinery and Intelligence. Mind 49: 433-460.\nCOMPUTING MACHINERY AND INTELLIGENCE\nBy A. M. Turing\n1. The Imitation Game\nI propose to consider the question, "Can machines think?" This should begin with\ndefinitions of the meaning of the terms "machine" and "think." The definitions might be\nframed so as to reflect so far as possible the normal use of the words, but this attitude is\ndangerous, If the meaning of the words "machine" and "think" are to be found by\nexamining how they are commonly used it is difficult to escape the conclusion that the\nmeaning and the answer to the question, "Can machines think?" is to be sought in a\nstatistical survey such as a Gallup poll. But this is absurd. Instead of attempting such a\ndefinition I shall replace the question by another, which is closely related to it and is\nexpressed in relatively unambiguous words.\nThe new form of the problem can be described in terms of a game which we call the\n\'imitation game." It is played with three people, a man (A), a woman (B), and an\ninterrogator (C) who may be of either sex. The interrogator stays in a room apart front the\nother two. The object of the game for the interrogator is to determine which of the other\ntwo is the man and which is the woman. He knows them by labels X and Y, and at the\nend of the game he says either "X is A and Y is B" or "X is B and Y is A." The\ninterrogator is allowed to put questions to A and B thus:\nC: Will X please tell me the length of his or her hair?\nNow suppose X is actually A, then A must answer. It is A\'s object in the game to try and\ncause C to make the wrong identification. His answer might therefore be:\n"My hair is shingled, and the longest strands are about nine inches long."\nIn order that tones of voice may not help the interrogator the answers should be written,\nor better still, typewritten. The ideal arrangement is to have a teleprinter communicating\nbetween the two rooms. Alternatively the question and answers can be repeated by an\nintermediary. The object of the game for the third player (B) is to help the interrogator.\nThe best strategy for her is probably to give truthful answers. She can add such things as\n"I am the woman, don\'t listen to him!" to her answers, but it will avail nothing as the man\ncan make similar remarks.\nWe now ask the question, "What will happen when a machine takes the part of A in this\ngame?" Will the interrogator decide wrongly as often when the game is played like this as\nhe does when the game is played between a man and a woman? These questions replace\nour original, "Can machines think?\n                 '}]

In [30]:
function_definition = [{
    'type': 'function',
    'function': {
        'name': 'extract_job_info',
        'description': 'Get the job information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'job': {'type': 'string', 'description': 'Job title'},
                'location': {'type': 'string', 'description': 'Location'}
            }
        }
    }
}]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=message_listings,
    tools = function_definition,
)

print(response.choices[0].message.content)

This excerpt is from Alan Turing's seminal paper, "Computing Machinery and Intelligence," where he introduces The Imitation Game, now commonly known as the Turing Test. Turing explores the question of whether machines can think by framing it in terms of a game involving a human interrogator, a man, and a woman. 

The game challenges the interrogator to distinguish between the two players based solely on their responses to questions. Turing suggests that if a machine can successfully imitate a human in this context, it raises the question of whether we should attribute the capacity to think to machines.

If you have specific questions about the text or need further analysis or explanations on certain aspects, feel free to ask!


### Parallel Function Calls


Respond to a review

In [54]:
import openai
import json

In [55]:
client = OpenAI(api_key=de_api_key)

In [56]:
function_definition = [
    {
        'type': 'function',
        'function': {
            'name': 'extract_sentiment_and_product_features',
            'parameters': {
                'type': 'object',
                'properties': {
                    'product': {
                        'type': 'string',
                        'description': 'The product name'
                    },
                    'sentiment': {
                        'type': 'string',
                        'description': 'The overall sentiment of the review'
                    },
                    'features': {
                        'type': 'array',
                        'items': {
                            'type': 'string'
                        },
                        'description': 'List of features mentioned in the review'
                    },
                    'suggestions': {
                        'type': 'array',
                        'items': {
                            'type': 'string'
                        },
                        'description': 'Suggestions for improvement'
                    }
                }
            }
        }
    }
]

In [57]:
function_definition.append(
    {
        'type': 'function',
        'function': {
            'name': 'reply_to_review',
            'description': 'Reply politely to the customer who wrote the review',
            'parameters': {
                'type': 'object',
                'properties': {
                    'reply': {
                        'type': 'string',
                        'description': 'Reply to post in response to the review'
                    }
                }
            }
        }
    }
)


In [58]:
messages = [
    {'role': 'system', 'content': 'Apply both functions and return responses.'},
    {'role': 'user', 'content': "I recently purchased the TechCorp ProMax and I'm absolutely in love with its powerful processor. However, I think they could really improve the product by deciding to offer more color options."}
]

In [59]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Don't make up values to fill the response with."})
messages.append({"role": "system", "content": "Ask for clarification if needed."})

In [63]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools = function_definition,
    tool_choice = {
        "type": "function",
        "function": {
            "name": "reply_to_review",
        }
    }
)

print(response.choices[0].message.tool_calls[0].function.arguments)

{"reply":"Thank you for your review! We appreciate your feedback and are glad you took the time to share your experience with us."}


#### Avoiding Inconsistency in replies

In [64]:
messages = [
    {'role': 'system', 'content': 'Apply both functions and return responses.'},
    {'role': 'user', 'content': 'Thrilled with the quality, but I think it should come with a wider choice of screen sizes.'}
]


In [65]:
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Don't make up values to fill the response with."})


In [69]:
def get_response(messages, function_definition):
  response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages,
      tools=function_definition
  )
  return str(response.choices[0].message.tool_calls[0].function.arguments) + ",\n " + str(response.choices[0].message.tool_calls[1].function.arguments)


In [70]:
response = get_response(messages, function_definition)

print(response)

{"sentiment": "Thrilled", "features": ["Quality"], "suggestions": ["Wider choice of screen sizes"]},
 {"reply": "Thank you for your feedback! We're thrilled to hear you are happy with the quality. We appreciate your suggestion for more screen size options and will take it into consideration."}


## Calling External APIs

In [76]:
import requests

def get_artwork(keyword):
    url = "https://api.artic.edu/api/v1/artworks/search"
    querystring = {"q": keyword}
    response = requests.request("GET", url, params=querystring)
    return response.text


In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are an AI assistant, a specialist in history of art. You should interpret the user prompt, and based on it extract one keyword for recommending artwork related to their preference."
    },
    {
        "role": "user",
        "content": "I don't have much time to visit the museum and would like some recommendations. I like the seaside and quiet places."
    }
]


In [77]:
function_definition = [
    {
        "type": "function",
        "function": {
            "name": "get_artwork",
            "description": "This function calls the Art Institute of Chicago API to find artwork that matches a keyword",
            "parameters": {
                "type": "object",
                "properties": {
                    "artwork keyword": {
                        "type": "string",
                        "description": "The keyword to be passed to the get_artwork function."
                    }
                }
            },
            "result": {
                "type": "string"
            }
        }
    }
]


In [82]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools = function_definition,
    tool_choice='auto'
)

In [83]:
import json

if response.choices[0].finish_reason == 'tool_calls':
    function_call = response.choices[0].message.tool_calls[0].function
    if function_call.name == "get_artwork":
        artwork_keyword = json.loads(function_call.arguments)["artwork keyword"]
        artwork = get_artwork(artwork_keyword)
        if artwork:
            print(f"Here are some recommendations: {[i['title'] for i in json.loads(artwork)['data']]}")
        else:
            print("Apologies, I couldn't make any recommendations based on the request.")
    else:
        print("Apologies, I couldn't find any artwork.")
else:
    print("I am sorry, but I could not understand your request.")


Here are some recommendations: ['Life-Size Black Bass', 'Figure Screen (Duein Fubara)', 'Screen', 'Marilyn Monroe (Marilyn)', 'Woman at an Easel (Green Screen)', 'Mao', 'Marilyn Monroe (Marilyn)', 'Fire Screen', 'Screen', 'Flowering Cherry and Autumn Maples with Poem Slips']


### Evaluations

- prompt injections


elaluation libraries and datasets

![Alt text](./Users/dylan/Documents/AIEng1/Screenshot%202024-07-20%20at%2012.58.49.png)